<a href="https://colab.research.google.com/github/charudatta07/datasharing/blob/master/Indian_Stock_Fundamental_DataSet_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from google.colab import drive
from google.colab import Import
from google.colab import Yahoo

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_name in numeric_columns:
        df[column_name] = df[column_name].str.replace(',', '') # Remove the thousands separator

        df[column_name] = df[column_name].astype(np.float64) # Convert the column to
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

stocks = ['SHEMAROO.NS','IRB.NS']
for symbol in stocks:
  df_balance_sheet=scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
  df_balance_sheet =df_balance_sheet.append(df_balance_sheet,sort=True)
  df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
  df_income_statement =df_income_statement.append(df_income_statement,sort=True)
  print(symbol)

#---------------------------------------------
df_balance_sheet.to_csv("df_balance_sheet.csv")
uploaded = drive.CreateFile({'Balance Sheet': 'Shemaroo Balance Sheet.csv'})
uploaded.SetContentFile("df_balance_sheet.csv")
uploaded.Upload()
#--------------------------------------------------
#---------------------------------------------
df_income_statement.to_csv("df_income_statement.csv")
uploaded = drive.CreateFile({'income statement ': 'Shemaroo income statement.csv'})
uploaded.SetContentFile("df_income_statement.csv")
uploaded.Upload()
#--------------------------------------------------
files.download("df_income_statement.csv")
files.download("df_balance_sheet.csv")
#------------------------------------------
df_income_statement
#df_balance_sheet


SHEMAROO.NS
IRB.NS


,Basic,Cost of Revenue,Date,Diluted,EBITDA,Gross Profit,Income Before Tax,Income Tax Expense,Income from Continuing Operations,Interest Expense,Net Income,Net Income available to common shareholders,Operating Income or Loss,Selling General and Administrative,Total Operating Expenses,Total Revenue
1,NaN,34862653.0,ttm,NaN,NaN,37756693.0,14377775.0,6040540.0,8337235.0,13148553.0,8337235.0,8337235.0,25943203.0,NaN,11813490.0,72619346.0
2,351450.0,31199930.0,3/31/2019,351450.0,31032480.0,35870250.0,14733240.0,6233590.0,8499650.0,10904110.0,8499650.0,8499650.0,24040290.0,653420.0,11829960.0,67070180.0
3,351450.0,23965590.0,3/31/2018,351450.0,29747220.0,32975400.0,14640090.0,5443520.0,9196570.0,9666680.0,9196570.0,9196570.0,21487730.0,936600.0,11487670.0,56940990.0
4,351450.0,22866800.0,3/31/2017,351450.0,31649180.0,35592560.0,9839430.0,2685220.0,7154210.0,13261850.0,7154740.0,7154740.0,22025480.0,437990.0,13567080.0,58459360.0
5,351450.0,20539754.0,3/31/2016,351450.0,27845498.0,30762560.0,8678668.0,2315952.0,6362716.0,10633395.0,6358223.0,6358223.0,18139751.0,465797.0,12622809.0,51302314.0
1,NaN,34862653.0,ttm,NaN,NaN,37756693.0,14377775.0,6040540.0,8337235.0,13148553.0,8337235.0,8337235.0,25943203.0,NaN,11813490.0,72619346.0
2,351450.0,31199930.0,3/31/2019,351450.0,31032480.0,35870250.0,14733240.0,6233590.0,8499650.0,10904110.0,8499650.0,8499650.0,24040290.0,653420.0,11829960.0,67070180.0
3,351450.0,23965590.0,3/31/2018,351450.0,29747220.0,32975400.0,14640090.0,5443520.0,9196570.0,9666680.0,9196570.0,9196570.0,21487730.0,936600.0,11487670.0,56940990.0
4,351450.0,22866800.0,3/31/2017,351450.0,31649180.0,35592560.0,9839430.0,2685220.0,7154210.0,13261850.0,7154740.0,7154740.0,22025480.0,437990.0,13567080.0,58459360.0
5,351450.0,20539754.0,3/31/2016,351450.0,27845498.0,30762560.0,8678668.0,2315952.0,6362716.0,10633395.0,6358223.0,6358223.0,18139751.0,465797.0,12622809.0,51302314.0
